# Задание №1

Есть Pandas DataFrame со столбцами `[“customer_id”, “product_id”, “timestamp”]`, который содержит данные по просмотрам товаров на сайте. Есть проблема – просмотры одного `customer_id` не разбиты на сессии (появления на сайте). Мы хотим разместить сессии так, чтобы сессией считались все смежные просмотры, между которыми не более 3 минут.

Написать методом который создаст в `Pandas DataFrame` столбец `session_id` и проставит в нем уникальный `int id` для каждой сессии.

У каждого пользователя может быть по несколько сессий. Исходный `DataFrame` может быть большим – до 100 млн строк.

# Решение

основные импорты

In [150]:
import pandas as pd
import numpy as np
import datetime
import random

генерируем массивы уникальных uuid для покупателей и товаров

In [151]:
import uuid

customers = []
max_unique_customers = 10
for _ in range(max_unique_customers):
    customers.append(str(uuid.uuid4()))
    
products = []
max_unique_products = 10
for _ in range(max_unique_products):
    products.append(uuid.uuid4().hex)

создаем Pandas DataFrame со столбцами [“customer_id”, “product_id”, “timestamp”], который содержит данные по просмотрам товаров на сайте

In [152]:
matrix = []
max_size = 100

dt = datetime.datetime(2022, 9, 1)
step = datetime.timedelta(seconds=5)

for _ in range(max_size):
    matrix.append([random.choice(customers), random.choice(products), dt])
    dt += step

In [153]:
columns = ["customer_id", "product_id", "timestamp"]

df = pd.DataFrame(data=matrix, columns=columns)
df

,customer_id,product_id,timestamp
0,7aec5821-c826-4411-acf1-b62b288b6430,89843c3b45aa484589355a62a2097f3c,2022-09-01 00:00:00
1,29af7157-b88c-4eec-bb74-29503415f3ff,08ae86e6c0c84f6e803b28c790d3dc0b,2022-09-01 00:00:05
2,fb42a385-e90a-4cc0-8e39-de84f5c375b7,55a1b7e3423f48a68c3affd7506befc2,2022-09-01 00:00:10
3,f783332d-3e6d-4fce-942f-822c3c00f948,89843c3b45aa484589355a62a2097f3c,2022-09-01 00:00:15
4,f783332d-3e6d-4fce-942f-822c3c00f948,e8baade2eb4b4681829f6dfd06e6f949,2022-09-01 00:00:20
...,...,...,...
95,f783332d-3e6d-4fce-942f-822c3c00f948,0e75efd18a344b979778162215488861,2022-09-01 00:07:55
96,7aec5821-c826-4411-acf1-b62b288b6430,0e75efd18a344b979778162215488861,2022-09-01 00:08:00
97,7aec5821-c826-4411-acf1-b62b288b6430,89843c3b45aa484589355a62a2097f3c,2022-09-01 00:08:05
98,519b3d5b-ab18-495e-93ff-83d23c0237b6,0ebacc8460e64db8a68bad4563efdc4b,2022-09-01 00:08:10


проверяем дубликаты для покупателей и продуктов

In [154]:
dups_customer_id = df.pivot_table(columns=['customer_id'], aggfunc='size')
dups_customer_id

customer_id
19a72154-aa2c-4d82-9515-b65e60bc9c99     7
29af7157-b88c-4eec-bb74-29503415f3ff    10
519b3d5b-ab18-495e-93ff-83d23c0237b6     7
69b78e0f-0ef8-4f7b-8c33-67b86fd99501    14
762472ac-c4f6-4694-b170-cd475145c4e2     9
7aec5821-c826-4411-acf1-b62b288b6430    13
b2e53b2e-3075-4a53-9289-cb622c5f276b     9
bf055c49-29e4-463b-a2cd-87c9e94434a7    13
f783332d-3e6d-4fce-942f-822c3c00f948     9
fb42a385-e90a-4cc0-8e39-de84f5c375b7     9
dtype: int64

In [155]:
dups_product_id = df.pivot_table(columns=['product_id'], aggfunc='size')
dups_product_id

product_id
018deebd90ae4314a5a4bc7245321dd9     6
08ae86e6c0c84f6e803b28c790d3dc0b    15
0e75efd18a344b979778162215488861    13
0ebacc8460e64db8a68bad4563efdc4b    11
55a1b7e3423f48a68c3affd7506befc2     5
6815fa3d5e56413b91ab03ca09bf5737    14
89843c3b45aa484589355a62a2097f3c    10
cb722602d3a34934a49c20b565b2a944     6
d91cae7b7f05407bb771b2ee133f8ed3     7
e8baade2eb4b4681829f6dfd06e6f949    13
dtype: int64

---------------------

In [173]:



def f(group):
    group['deltaTsec'] = group["timestamp"].diff().dt.seconds.fillna(0).astype(int)
#     print(group)
    
    sessions = []
    session_time = 0
    session_limit = 180
    i = 0
    for delta in group['deltaTsec']:
        session_time += delta
        if i == 0 or session_time > session_limit:
            session_time = 0
            sessions.append(str(uuid.uuid4()))
        elif session_time <= session_limit:
            sessions.append(sessions[i - 1])
        
        print("session_time",session_time)
        i += 1
        
    group['session_id'] = sessions
    print(group)
    return group

df_groupby = df.groupby("customer_id", as_index=False).apply(f)
df_groupby

session_time 0
session_time 70
session_time 130
session_time 0
session_time 160
session_time 0
session_time 40
                             customer_id                        product_id  \
9   19a72154-aa2c-4d82-9515-b65e60bc9c99  55a1b7e3423f48a68c3affd7506befc2   
23  19a72154-aa2c-4d82-9515-b65e60bc9c99  89843c3b45aa484589355a62a2097f3c   
35  19a72154-aa2c-4d82-9515-b65e60bc9c99  e8baade2eb4b4681829f6dfd06e6f949   
48  19a72154-aa2c-4d82-9515-b65e60bc9c99  d91cae7b7f05407bb771b2ee133f8ed3   
80  19a72154-aa2c-4d82-9515-b65e60bc9c99  08ae86e6c0c84f6e803b28c790d3dc0b   
85  19a72154-aa2c-4d82-9515-b65e60bc9c99  d91cae7b7f05407bb771b2ee133f8ed3   
93  19a72154-aa2c-4d82-9515-b65e60bc9c99  e8baade2eb4b4681829f6dfd06e6f949   

             timestamp  deltaTsec                            session_id  
9  2022-09-01 00:00:45          0  35b1ded5-2b8f-4eb0-8451-69dcfb1a02d9  
23 2022-09-01 00:01:55         70  35b1ded5-2b8f-4eb0-8451-69dcfb1a02d9  
35 2022-09-01 00:02:55         60  35b1ded

,customer_id,product_id,timestamp,deltaTsec,session_id
0,7aec5821-c826-4411-acf1-b62b288b6430,89843c3b45aa484589355a62a2097f3c,2022-09-01 00:00:00,0,57da5b4e-a224-43f2-9793-1d04bed18cfe
1,29af7157-b88c-4eec-bb74-29503415f3ff,08ae86e6c0c84f6e803b28c790d3dc0b,2022-09-01 00:00:05,0,1e474b63-204f-4812-b7e7-e8fa1d223ea2
2,fb42a385-e90a-4cc0-8e39-de84f5c375b7,55a1b7e3423f48a68c3affd7506befc2,2022-09-01 00:00:10,0,8a5c2868-0b4d-4503-9364-bb26537cdbda
3,f783332d-3e6d-4fce-942f-822c3c00f948,89843c3b45aa484589355a62a2097f3c,2022-09-01 00:00:15,0,736c57e1-7ea4-4184-9589-eb5fa2ad178e
4,f783332d-3e6d-4fce-942f-822c3c00f948,e8baade2eb4b4681829f6dfd06e6f949,2022-09-01 00:00:20,5,736c57e1-7ea4-4184-9589-eb5fa2ad178e
...,...,...,...,...,...
95,f783332d-3e6d-4fce-942f-822c3c00f948,0e75efd18a344b979778162215488861,2022-09-01 00:07:55,60,93744fc3-d261-4a1d-a5ae-b83ba8fc67ad
96,7aec5821-c826-4411-acf1-b62b288b6430,0e75efd18a344b979778162215488861,2022-09-01 00:08:00,30,038552e0-c5e2-4474-a204-a92c94d15135
97,7aec5821-c826-4411-acf1-b62b288b6430,89843c3b45aa484589355a62a2097f3c,2022-09-01 00:08:05,5,038552e0-c5e2-4474-a204-a92c94d15135
98,519b3d5b-ab18-495e-93ff-83d23c0237b6,0ebacc8460e64db8a68bad4563efdc4b,2022-09-01 00:08:10,70,7a689ffe-3fb5-41e0-9767-4c73b16485e4


In [174]:
dups_session_id = df_groupby.pivot_table(columns=['session_id'], aggfunc='size')
dups_session_id

session_id
038552e0-c5e2-4474-a204-a92c94d15135    3
053a5c04-7a44-4f7c-a589-bc20aa402df3    2
1e474b63-204f-4812-b7e7-e8fa1d223ea2    4
2fd3de21-2b77-4473-b917-e95a05b723d3    4
35b1ded5-2b8f-4eb0-8451-69dcfb1a02d9    3
44cc5c11-ce21-4b6a-96bd-77f3e02a772e    3
4663458b-6323-4149-86c6-1cc721fdb071    5
4e1bbf58-b1cb-4b16-a375-6d36805d0cc6    6
57da5b4e-a224-43f2-9793-1d04bed18cfe    4
63d31967-a6ef-4f39-9658-fd767d5289fc    4
63e45a53-a63c-41a6-9208-a954a335e93a    2
7345634a-cc5d-43d1-8025-58da4daeed47    4
736c57e1-7ea4-4184-9589-eb5fa2ad178e    5
75cf4ddd-e0d1-4bf1-8760-6874947b9fa9    8
7693275b-11c8-47c0-be28-f36e95ef7371    2
7a689ffe-3fb5-41e0-9767-4c73b16485e4    1
8154dcb8-a3a5-43be-8351-00dc4058682f    1
8a5c2868-0b4d-4503-9364-bb26537cdbda    5
93744fc3-d261-4a1d-a5ae-b83ba8fc67ad    2
a4be0fe0-9632-4109-9458-d8a1ad86f485    7
d301a21f-4965-4287-9e6c-fc14b25d013d    5
e195ca8d-21f8-48da-abcf-7c2f01a83376    6
e865e083-e0b9-4182-a27e-49b5b6fb6529    2
e9a24480-e40f-4d43-9ded